## 参考資料
- [詳細確率ロボティクスのgit](https://github.com/ryuichiueda/LNPR_BOOK_CODES)
---
## ベイズフィルタによる自己位置推定
- 自己位置推定に使える情報
  - $x_0$:最初の姿勢
  - $u_{1:t}$:各時刻の制御指令
  - $z_{1:t}$:各時刻のセンサ値
  - モデル
    - $p(x|x_{t-1},u_t)$:状態遷移モデル
      - 1個前の姿勢$x_{t-1}$と今回の制御指令$u_t$から状態(姿勢)がどう更新されるかの確率分布
    - $p(z|x)$:観測モデル
      - 現在の姿勢からわかるセンサ値の確率分布？
- 計算したい確率分布
  - $p_t(x|x_0,u_{t:1},z_{t:1})$
    - 初期姿勢、これまで与えた制御指令、得られたセンサ値→現在のロボットの姿勢の確率分布を求めたい
    - これを**信念(belief)**とよび、ロボットの信念分布を$b_t(x)$と定義する
- 信念の計算
  - 初期信念$b_0$
    - 今回は分かっているという前提で、とがった確率分布を考える
  - 情報による信念の更新
    - ロボットの移動による更新
      - 時刻t-1からtまでロボットが$u_t$で動くとき、信念分布も$b_{t-1}$から変化
        - $b_{t-1}(x)\rightarrow b_{t-1}(x_0,u_{1:t},z_{1:t-1})=\hat{b_t}$
        - $\hat{b_t}$は$b_t$と違って$z_t$の情報がまだない(zがt:t-1になっていることに注意)
      - 得られた信念分布を積分したものが最終的な信念分布になる
        - $x'$は移動前の分布におけるある姿勢
        - $\hat{b_t}=\int_{/{x'\in{\chi}}}p(x|x',u_t)b_{t-1}(x')dx'$
          - 移動前のある姿勢X'の信念分布$b_{t-1}(x')$に、移動後の分布の確率密度$p(x|x',u_t)$をかける。これを移動前のある姿勢x'の全部について積分
          - ポイントは、移動前も姿勢は分布をもっていて(そのうちの一つがx')、移動前の分布に確率密度をかけて積分したものを移動後の分布と考える
        - $\hat{b_t}=<p(x|x',u_t)>_{b_{t-1}(x')}$
          - 移動前のある姿勢の分布$b_{t-1}(x')$に確率密度の期待値をかけたものとも解釈できる
            - どうせ移動前の1つの分布から移動後の1つの分布ができるだけだから、いちいち積分しないで確率密度の期待値としてまとめて、一括で移動後の分布を計算
        - 一般に使われる決まった名前はないが「マルコフ連鎖の式」とか「状態遷移の式」と呼ぶこととする
          - マルコフ性：次の状態は直前の姿勢と制御出力からしか決まらない、それより前の情報は不要
    - ランドマークの観測による更新
      - $\hat{b_t}(x)$に$z_t$の情報を追加
        - $\hat{b_t}(x)\rightarrow \hat{b_t}(x|z_t)=b_t(x)$
        - ベイズの定理から、各センサ値の掛け算によって分布を計算できる
          - $b_t(x)=\hat{b_t}(x|z_t)=\eta p(z_t|x) \hat{b_t}(x)$
        - なお、各センサ値の独立性を仮定すると
          - $b_t(x)=\eta \hat{b_t}(x) \prod_{j=0}^{N_m-1} p_j(z_{j, t}|x)$
        - センサ値がない場合は$\hat{b_t}(x)=b_t(x)$
    - ベイズフィルタ
      - まとめると、2つの式で信念の更新ができる
      - 移動時：$\hat{b_t}=<p(x|x',u_t)>_{b_{t-1}(x')}$
      - 観測時：$b_t(x)=\eta p(z_t|x) \hat{b_t}(x)$
      - これをベイズフィルタという
      - では、これをどうやって実装するのか？
        - その具体的なやり方を今後扱っていく

## パーティクルフィルタによる自己位置推定
- ベイズフィルタについて、ここではパーティクルフィルタを使って実装しする
  - パーティクルの集合で信念分布を表現
  - パーティクル：ロボットの分身だとここでは考える
  - 

In [7]:
# conect to drive on colab
# from google.colab import drive
# drive.mount("/content/drive")
# dir_path="./drive/MyDrive/Colab Notebooks/ProbabilisticRobotics/"
dir_path="./"

import sys
sys.path.append(dir_path)
from robot import *
from scipy.stats import expon,norm,uniform
import numpy as np
pi=np.pi

In [13]:
class EstimationAgent(Agent):
    def __init__(self,nu,omega,estimator):
        super().__init__(nu,omega)
        self.estimator=estimator
        
    def draw(self,ax,elems):
        self.estimator.draw(ax,elems)
        elems.append(ax.text(0,0,"hoge",fontsize=10))

In [9]:
class Particle:
    def __init__(self,init_pose):
        self.pose=init_pose



In [14]:
class Mcl:
    def __init__(self,init_pose,num) -> None:
        self.particles=[Particle(init_pose) for i in range(num)]
    def draw(self,ax,elems):
        xs=[p.pose[0] for p in self.particles]
        ys=[p.pose[1] for p in self.particles]
        vxs=[np.cos(p.pose[2]) for p in self.particles]
        vys=[np.sin(p.pose[2]) for p in self.particles]
        elems.append(ax.quiver(xs,ys,vxs,vys,color="blue",alpha=0.5))

In [15]:

world = World(10, 0.1)


m = Map()                                  
m.append_landmark(Landmark(-4,2))
m.append_landmark(Landmark(2,-3))
m.append_landmark(Landmark(3,3))
world.append(m)     

init_pose=np.array([2, 2, pi/6]).T
estimator=Mcl(init_pose,100)
circling=EstimationAgent(0.2,10/180*pi,estimator)
r= Robot( init_pose, sensor=Camera(m), agent=circling)
world.append(r)

world.draw()